In [1]:
import preprocessing as mypre
import features_extraction as myfe

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [2]:
X, y = mypre.read_dataset(pth='./dataset64/')
print(X.shape, y.shape)

(922, 64, 64, 3) (922,)


In [3]:
for x in X:
    x = mypre.fix_illumination(mypre.fix_contrast(x))
print(X.shape)    

C:\Users\Ghieath\AppData\Roaming\Python\Python310\site-packages\skimage\_shared\utils.py:394: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel, or set channel_axis.
  return func(*args, **kwargs)


(922, 64, 64, 3)


In [4]:
HOG=0
LBP=0
GLCM=0
COLOR_HISTOGRAM=1
COLOR_MOMENTS=0
COLOR_CORRELOGRAM=0
GABOR=0
WATERSHED=0
CANNY=0
SOBEL=0
#
args = (HOG, LBP, GLCM, COLOR_HISTOGRAM, COLOR_MOMENTS, COLOR_CORRELOGRAM,
        GABOR, WATERSHED, CANNY, SOBEL)

In [5]:
X_features = [myfe.extract_features(x, *args) for x in X]
X_features = np.array(X_features)
print(X_features.shape)

(922, 64)


In [7]:
hoooooooooooooooooooooooooooooooooooooooooool up

In [8]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [9]:
# macro f1 import
from sklearn.metrics import f1_score

# Train model
def test_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    name = str(model).split('(')[0]
    # print(name, f1_score(y_test, y_pred, average='macro').round(3))
    f1 = np.round(f1_score(y_test, y_pred, average='macro'), 3) *100
    f1 = str(f1)[:4]
    print(f1, end=' |')

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


for clf in [
            # SVC(kernel='linear', C=1.0, random_state=42), 
            # GaussianNB(),
            # RandomForestClassifier(n_estimators=100, max_depth=2, random_state=42),
            XGBClassifier(
                booster='gbtree',
                learning_rate=0.3,
                max_depth=3,
                # objective="multi:softprob",
                random_state=42,
                # num_class=2,
                # eval_metric="auc",
                # eval_metric="mlogloss",
            ),
            # AdaBoostClassifier(n_estimators=100, random_state=42),
            # DecisionTreeClassifier(random_state=42)
            ]:
    test_model(clf, X_train, y_train, X_test, y_test)

80.5 |

In [11]:
# Random Search CV
from sklearn.model_selection import RandomizedSearchCV
# Grid Search CV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Create the parameter grid based on the results of random search
# XGBClassifier
param_grid = {
    'learning_rate': [0.1, 0.3, 0.5, 0.7, 1],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 200, 300, 400, 500],
    'booster': ['gbtree', 'gblinear', 'dart'],
    'objective': ['multi:softprob'],
    'random_state': [42],
    'num_class': [2],
    'eval_metric': ['auc', 'mlogloss'],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
}

# Create a based model
model = XGBClassifier()
# Instantiate the grid search model
grid_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid,
                            cv = 3, n_jobs = -1, verbose = 2, scoring=make_scorer(f1_score, average='macro'))

# # Grid search
# grid_search = GridSearchCV(estimator = model, param_grid = param_grid,
#                             cv = 3, n_jobs = -1, verbose = 2, scoring=make_scorer(f1_score, average='macro'))

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
print(grid_search.best_estimator_.score(X_test, y_test))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


# Results (macro F1):

Features Used | SVM | Naive Bayes | Random Forest | XGBoost | AdaBoost | Decision Tree | MLP
--- | --- | --- | --- | --- | --- | --- | --- |
hog, lbp, glcm, color_histogram, color_moments | 77.8 | 71.9 | 79.5 | 89.2 | 82.2

Features Used | SVM | XGBoost | AdaBoost
--- | --- | --- | ---
o
**64x64**
hog | 71.8 |77.8 |72.8
LBP | 32. |69.7 |67.6
GLCM | 69.1 |72.9 |71.5
Color Histogram | 68.6 |75.6 |74.6
Color Moments (lab) | 71.5 |81. |73.9
Collerogram | 49.5 |74.0 |73.5 |
Gabor Filter | 53.9 |67.5 |61.6 |
WaterShed | 32.0 |53.5 |47.9
Moments (lab), LBP | 69.9 |81.6 |76.0
Moments (lab), hog | 73.5 |82.1 |78.9
Moments (lab), GLCM | 75.1 |82.6 |75.0
Moments, Hist | 82.1 |**85.9** |82.7
Correlogram, gabor, watershed | 63.2 |76.1 |73.3
Correlogram, gabor | 63.2 |77.8 |76.7 |
o
224x224 | ||
Hist | 70.7 |78.9 |74.4
GLCM | 71.7 |64.8 |67.5
Moments | 75.9|82.6 |82.6
Moments, Hist | 79.5 |83.2 |81.5
hog,lbp,glcm,histo,moments,watershed | 78.9 | 86.5 | TLE
hog,lbp,glcm,hist,mom,correl,gab | 78.9 | 87.5 | 87
o
tuning XGBoost
hog,glcm,hist,mom,correl | | 88.6 |
hog,glcm,hist,mom | | 82.1 |
glcm,hist,mom,correl | | 90.2 |
lbp,glcm,hist,mom,correl | |  91.3 |
lbp,glcm,hist,mom,correl,gabor | | 90.8 |
lbp,hist,mom,correl | | 91.3 |
lbp,mom,correl | | 90.3 |
lbp,correl | | 89.7 |
o
a | kernel='rbf', C=50.0, gamma=1.88 | | n_estimators=400
lbp,hist,correl | rbf:91.9 | 92.4 | 91.9
lbp,glcm,hist,correl | | 92.4 |
lbp,hist,mom,correl | | **93** |
lbp,glcm,hist,mom,correl |  | **93** | 

Features Used | SVM | Naive Bayes | Random Forest | XGBoost | AdaBoost | Decision Tree
--- | --- | --- | --- | --- | --- | --- |
o 
**64x64**
hog | 71.8 |71.3 |75.1 |77.8 |72.8 |66.9 |58.7 | 73.9 |
LBP | 32. |55.8 |61.4 |69.7 |67.6 |53. |
GLCM | 69.1 |65.9 |68.1 |72.9 |71.5 |68.6 |70.
Color Histogram | 68.6 |73.1 |69.7 |75.6 |74.6 |65.4 |73.8
Color Moments (lab) | 71.5 |72.8 |70.9 |81. |73.9 |75.6 |
WaterShed | 32.0 |54.1 |58.4 |53.5 |47.9 |45.9 |
Moments (lab), LBP | 69.9 |73.6 |67.5 |81.6 |76.0 |78.9 |
Moments (lab), hog | 73.5 |71.3 |76.7 |82.1 |78.9 |70.7 |
Moments (lab), GLCM | 75.1 |73.2 |75.3 |82.6 |75.0 |81.0 |
Moments, Hist | 82.1 |73.5 |82.5 |**85.9** |82.7 |76.8 |
o
224x224 | 
Hist | 70.7 |70.5 |73.2 |78.9 |74.4 |70.3 |
GLCM | 71.7 |70.3 |70.7 |64.8 |67.5 |65.9 |
Moments | 75.9| 73.6 |76.6 |82.6 |82.6 | 75.9
Moments, Hist | 79.5 ||75.9 |83.2 |81.5 |

